In [2]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [14]:
# use first 500,000 rows for training and testing
data = pd.read_csv('train.csv',nrows=500000)
train_data = data.iloc[:450000] # use first 450,000 obs. for training
test_data = data.iloc[450000:] # use last 50,000 obs for testing

In [17]:
# get train and test target from their respective datasets
train_target = train_data.pop('target')
test_target = test_data.pop('target')

In [18]:
# remove id columns from datasets
train_row_id = train_data.pop('row_id')
train_time_id = train_data.pop('time_id')
train_investment_id = train_data.pop('investment_id')

test_row_id = test_data.pop('row_id')
test_time_id = test_data.pop('time_id')
test_investment_id = test_data.pop('investment_id')

# Deep Neural Network

In [22]:
# import DNN libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras import Sequential 
from scipy.stats import pearsonr # pearson correlation coef

In [34]:
# create DNN 
def create_dnn(nodes,layers,dropout,activation):
    model = Sequential()
    
    for i in list(range(1,layers+1)):
        if i == 1:
            model.add(Dense(nodes, input_dim = (train_data.shape[1]), activation = 'swish'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))
        else:
            model.add(Dense(i/2, activation='swish'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))

    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mae'])
    return model

In [37]:
models = {}
for i in [512,256,128]:
    for j in [5,4,3]:
                model = create_dnn(i,j,0.3,'swish') # create a DNN with the given number of initial nodes and layers
            
                model.fit(train_data, train_target, epochs = 40, batch_size = 4096, verbose = 1) # fit the DNN
                
                comb = str(i)+'-'+str(j)
                models[comb] = model # store the model

Epoch 1/40
110/110 [==============================] - 6s 58ms/step - loss: 2.2331 - mae: 0.8520
Epoch 2/40
110/110 [==============================] - 6s 58ms/step - loss: 1.6926 - mae: 0.8052
Epoch 3/40
110/110 [==============================] - 6s 58ms/step - loss: 1.4158 - mae: 0.7747
Epoch 4/40
110/110 [==============================] - 6s 58ms/step - loss: 1.2270 - mae: 0.7482
Epoch 5/40
110/110 [==============================] - 7s 60ms/step - loss: 1.1020 - mae: 0.7256
Epoch 6/40
110/110 [==============================] - 7s 61ms/step - loss: 1.0264 - mae: 0.7074
Epoch 7/40
110/110 [==============================] - 7s 60ms/step - loss: 0.9124 - mae: 0.6630
Epoch 8/40
110/110 [==============================] - 7s 61ms/step - loss: 0.8777 - mae: 0.6448
Epoch 9/40
110/110 [==============================] - 7s 61ms/step - loss: 0.8713 - mae: 0.6409
Epoch 10/40
110/110 [==============================] - 7s 61ms/step - loss: 0.8700 - mae: 0.6400
Epoch 11/40
110/110 [==================

In [120]:
# get PCC and p-value on test set for each model
pearson_coef = []
pearson_p = []
for i in models:
    pred = models[i].predict(test_data).reshape(50000,)
    targ = np.array(test_target).reshape(50000,).astype('float32')
    results = pearsonr(pred, targ)
    pearson_coef.append(results[0])
    pearson_p.append(results[1])

In [121]:
# show results
n = 0
for i in models:
    print("Model: ",i)
    print("Pearson Coef: ",pearson_coef[n])
    print("Pearson P-Value: ",pearson_p[n])
    n+=1
    
DNN_results = pd.DataFrame([pearson_coef,pearson_p],columns= models.keys(),index=["PCC","P-Value"])
DNN_results

Model:  512-5
Pearson Coef:  0.10435927508124318
Pearson P-Value:  4.398409931642727e-121
Model:  512-4
Pearson Coef:  0.12227179884800957
Pearson P-Value:  8.388047922672366e-166
Model:  512-3
Pearson Coef:  0.11576269055212128
Pearson P-Value:  1.0253749826401292e-148
Model:  256-5
Pearson Coef:  0.1192094740967064
Pearson P-Value:  1.208023395294987e-157
Model:  256-4
Pearson Coef:  0.1095912628794523
Pearson P-Value:  2.1293748299609102e-133
Model:  256-3
Pearson Coef:  0.12035871192711757
Pearson P-Value:  1.1098585533898039e-160
Model:  128-5
Pearson Coef:  0.049417309945315235
Pearson P-Value:  2.038422867454673e-28
Model:  128-4
Pearson Coef:  0.12771977811513616
Pearson P-Value:  7.645201554990986e-181
Model:  128-3
Pearson Coef:  0.1217218129144381
Pearson P-Value:  2.5383371265869945e-164


,512-5,512-4,512-3,256-5,256-4,256-3,128-5,128-4,128-3
PCC,1.043593e-01,1.222718e-01,1.157627e-01,1.192095e-01,1.095913e-01,1.203587e-01,4.941731e-02,1.277198e-01,1.217218e-01
P-Value,4.398410e-121,8.388048e-166,1.025375e-148,1.208023e-157,2.129375e-133,1.109859e-160,2.038423e-28,7.645202e-181,2.538337e-164


# Random Forest

In [127]:
from sklearn.ensemble import RandomForestRegressor
models_rf = {}


for i in [2,5,10]:
    model= RandomForestRegressor(max_depth=i,n_estimators=17)
    model.fit(train_data,train_target)
    comb = str(i)
    models_rf[comb] = model

In [128]:
pearson_coef_rf = []
pearson_p_rf = []
for i in models_rf:
    pred = models_rf[i].predict(test_data).reshape(50000,)
    targ = np.array(test_target).reshape(50000,).astype('float32')
    results = pearsonr(pred, targ)
    pearson_coef_rf.append(results[0])
    pearson_p_rf.append(results[1])

In [129]:
# show results
n = 0
for i in models_rf:
    print("Model: ",i)
    print("Pearson Coef: ",pearson_coef_rf[n])
    print("Pearson P-Value: ",pearson_p_rf[n])
    n+=1
    
rf_results = pd.DataFrame([pearson_coef_rf,pearson_p_rf],columns= models_rf.keys(),index=["PCC","P-Value"])
rf_results

Model:  2
Pearson Coef:  0.07259279055511453
Pearson P-Value:  2.1172311983616614e-59
Model:  5
Pearson Coef:  0.08145395648163131
Pearson P-Value:  2.3295223367972207e-74
Model:  10
Pearson Coef:  0.08946673155727483
Pearson P-Value:  2.2284561640876307e-89


,2,5,10
PCC,7.259279e-02,8.145396e-02,8.946673e-02
P-Value,2.117231e-59,2.329522e-74,2.228456e-89


# Ensemble

In [130]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

mod1 = GradientBoostingRegressor(n_estimators=10)
mod2 = RandomForestRegressor(n_estimators=10)
mod3 = LinearRegression()
ensemble = VotingRegressor(estimators=[('gb', mod1), ('rf', mod2), ('lr', mod3)])
ensemble_model = ensemble.fit(train_data, train_target)

In [158]:
pred = ensemble_model.predict(test_data).reshape(50000,)
targ = np.array(test_target).reshape(50000,).astype('float32')
results = pearsonr(pred, targ)
pearson_coef_en = results[0]
pearson_p_en = results[1]

# DNN - Improved

In [137]:
models_improved = {}
for i in [128,64,32]:
    for j in [5,4,3,2]:
                model = create_dnn(i,j,0.3,'swish') # create a DNN with the given number of initial nodes and layers

                model.fit(train_data, train_target, epochs = 100, batch_size = 4096, verbose = 1) # fit the DNN
                
                comb = str(i)+'-'+str(j)
                models_improved[comb] = model # store the model

Epoch 1/100
110/110 [==============================] - 2s 15ms/step - loss: 1.3485 - mae: 0.7957
Epoch 2/100
110/110 [==============================] - 2s 16ms/step - loss: 1.0815 - mae: 0.7369
Epoch 3/100
110/110 [==============================] - 2s 17ms/step - loss: 1.0219 - mae: 0.7085
Epoch 4/100
110/110 [==============================] - 2s 17ms/step - loss: 0.9803 - mae: 0.6914
Epoch 5/100
110/110 [==============================] - 2s 16ms/step - loss: 0.9522 - mae: 0.6784
Epoch 6/100
110/110 [==============================] - 2s 16ms/step - loss: 0.9315 - mae: 0.6685
Epoch 7/100
110/110 [==============================] - 2s 16ms/step - loss: 0.9152 - mae: 0.6618
Epoch 8/100
110/110 [==============================] - 2s 16ms/step - loss: 0.8967 - mae: 0.6539
Epoch 9/100
110/110 [==============================] - 2s 16ms/step - loss: 0.8776 - mae: 0.6441
Epoch 10/100
110/110 [==============================] - 2s 17ms/step - loss: 0.8728 - mae: 0.6411
Epoch 11/100
110/110 [=======

In [138]:
# get PCC and p-value on test set for each model
pearson_coef_imp = []
pearson_p_imp = []
for i in models_improved:
    pred = models_improved[i].predict(test_data).reshape(50000,)
    targ = np.array(test_target).reshape(50000,).astype('float32')
    results = pearsonr(pred, targ)
    pearson_coef_imp.append(results[0])
    pearson_p_imp.append(results[1])

In [139]:
pearson_coef_imp

[0.10743922196923479,
 0.11347369519451647,
 0.11492025134806846,
 0.11225180171056587,
 0.12389366078207467,
 0.11734464508021099,
 0.11614679820352401,
 0.1228070722234781,
 0.13433897056388372,
 0.1318593599667382,
 0.12806720395610785,
 0.12040990904751069]

# Save Results

In [164]:
# DNN
with open('dnn_keys.data', 'wb') as fi:
    pickle.dump(list(models.keys()), fi)
with open('dnn_coefs.data','wb') as fi:
    pickle.dump(list(pearson_coef),fi)
with open('dnn_p.data','wb') as fi:
    pickle.dump(list(pearson_p),fi)
    
# RF
with open('rf_keys.data', 'wb') as fi:
    pickle.dump(list(models_rf.keys()), fi)
with open('rf_coefs.data','wb') as fi:
    pickle.dump(list(pearson_coef_rf),fi)
with open('rf_p.data','wb') as fi:
    pickle.dump(list(pearson_p_rf),fi)
    
    
# DNN - Improved
with open('dnn_imp_keys.data', 'wb') as fi:
    pickle.dump(list(models_improved.keys()), fi)
with open('dnn_imp_coefs.data','wb') as fi:
    pickle.dump(list(pearson_coef_imp),fi)
with open('dnn_imp_p.data','wb') as fi:
    pickle.dump(list(pearson_p_imp),fi)

In [168]:
with open('dnn_imp_keys.data', 'rb') as fi:
   x = pickle.load(fi)
x

['128-5',
 '128-4',
 '128-3',
 '128-2',
 '64-5',
 '64-4',
 '64-3',
 '64-2',
 '32-5',
 '32-4',
 '32-3',
 '32-2']